# Residential Tree Planting project data prep

## Loading and formatting data

In [957]:
import pandas as pd
import os
import numpy as np
import geopy
import geopandas

In [958]:
#Rename excel files
old_names = list(os.listdir('./data'))

for i in old_names:
    if '.xlsx' not in i:
        old_names.remove(i)

old_names

['fall_2023.xlsx',
 'fall_2024.xlsx',
 'fall_2025.xlsx',
 'spring_2023.xlsx',
 'spring_2024_1.xlsx',
 'spring_2024_2.xlsx',
 'spring_2025.xlsx']

In [959]:
new_names = []
for i in old_names:
    new_name = i.lower().replace(" ","_").strip()
    new_name = new_name.replace("casey_", "").replace("tree_","").replace("trees_","").replace("planting_","")
    new_names.append(new_name)

new_names

['fall_2023.xlsx',
 'fall_2024.xlsx',
 'fall_2025.xlsx',
 'spring_2023.xlsx',
 'spring_2024_1.xlsx',
 'spring_2024_2.xlsx',
 'spring_2025.xlsx']

In [960]:
# ALREADY CHANGED NAMES THEREFORE NOT NECESSARY
# new_names[1] = 'spring_2023.xlsx'
# new_names[2] = 'spring_2024_2.xlsx'
# new_names[4] = 'fall_2024.xlsx'
# new_names[6] = 'spring_2024_1.xlsx'

# new_names

In [961]:
# ALREADY CHANGED NAMES THEREFORE NOT NECESSARY
# for i in old_names:
#     os.rename(i, new_names[old_names.index(i)])

In [962]:
# Read files into df
dfs = {}
for i in new_names:
    dfs[i] = pd.read_excel(("./data/"+i), header=0)

C:\Users\pmedina\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [963]:
dfs.keys()

dict_keys(['fall_2023.xlsx', 'fall_2024.xlsx', 'fall_2025.xlsx', 'spring_2023.xlsx', 'spring_2024_1.xlsx', 'spring_2024_2.xlsx', 'spring_2025.xlsx'])

## Formatting data

### Formatting functions

In [964]:
# Variable name formatting
def clean_column_names(df):
    df.columns = (df.columns
               .str.strip()
               .str.lower()
               .str.replace(" ", "_")
               .str.replace(";","")
               .str.replace("\\t","")
               .str.replace("[()€$]", "",

                            regex=True))
    return df

In [965]:
pd.set_option('mode.chained_assignment', None)
def update_species(df):
    species_columns = ['species_1', 'species_2', 'species_3', 'species_4']
    df_columns = list(df.columns)
    test_value = 0
    for name in species_columns:
        if name in df_columns:
            test_value += 1
    if test_value >= 3:
        return df
    else:
        for index, row in df.iterrows():
            if ',' in row['species_1']:
                species = df.loc[index, 'species_1'].split(',')
                species_1 = species[0]
                species_2 = species[1]
                df.loc[index, 'species_1'] = species_1
                df.loc[index, 'species_2'] = species_2
            elif '2' in row['species_1']:
                species = row['species_1'].strip().replace('2','').replace('()','')
                df.loc[index, 'species_1'] = species
                df.loc[index, 'species_2'] = species
            else:
                row['species_2'] = ''
        return df

In [966]:
def column_clean(df):
    del_words = ['application', 'name','email','casey','form','contact','area','inspection','date','phone']
    column_list = list(df.columns)
    columns_to_drop = []
    new_df = df

    if 'unnamed:_1' in column_list:
        new_column_names = list(new_df.loc[0,])
        new_df.columns = new_column_names
        new_df = clean_column_names(new_df)
        new_df = new_df.iloc[1:]
        column_list = list(new_df.columns)
        
    for column_name in column_list:
        for word in del_words:
            if word in column_name:
                if column_name not in columns_to_drop:
                    columns_to_drop.append(column_name)

    new_df.drop(columns=columns_to_drop, axis = 1, inplace = True)
     
    species_columns = 1
    
    for column_name in column_list:
        if "species" in column_name:
            # species_columns.append(column_name)
            new_name = 'species_' + str(species_columns)
            new_df.rename(columns={column_name:new_name}, inplace = True)
            species_columns += 1
        elif "district_#" in column_name or 'city_dist' in column_name:
            new_df.rename(columns={column_name:"city_district"}, inplace = True)
        elif "address" in column_name:
            new_df.rename(columns={column_name:"address"}, inplace = True)
        
            
    return new_df
            

In [967]:
def format_table(df, year):
    if 'unnamed:_1' in list(df.columns):
            new_df = column_clean(df)
            new_df = update_species(new_df)
            new_df['year'] = year
            return new_df
    else:
            new_df = clean_column_names(df)
            new_df = column_clean(new_df)
            new_df = update_species(new_df)
            new_df['year'] = year
            return new_df

In [968]:
for i in dfs:
    clean_column_names(dfs[i])

In [969]:
spring_2023 = dfs['spring_2023.xlsx']
fall_2023 = dfs['fall_2023.xlsx']
spring_2024_2 = dfs['spring_2024_2.xlsx']
fall_2025 = dfs['fall_2025.xlsx']
fall_2024 = dfs['fall_2024.xlsx']
spring_2025 = dfs['spring_2025.xlsx']
spring_2024_1 = dfs['spring_2024_1.xlsx']

## Format tables
### Spring 2023

In [970]:
spring_2023.head(2)

,application_date_planting_on_resid_property,area,district_#,first_and_last_name:,email_address:,phone_number:,address:,preferred_contact,consultation_date,casey_trees_date_planted,city_inspection_date,species_planted,notes_casey_trees
0,2022-05-14 21:50:25,Daniels Pk E,1,Kirstin Showalter,kirstinsd@gmail.com,7346492987,5121 Kenesaw St,Email,2023-02-28,NaN,NaN,"A.Hornbeam, Wil Oak",match
1,2022-11-11 13:10:39,Daniels Pk W,1,Anne Tolbert,ktolbertmd@gmail.com,301-613-3496,4908 Hollywood Rd,Email,2023-03-01,NaN,NaN,(2) Willow Oaks,no map


In [971]:
spring_2023 = format_table(spring_2023, 2023)
spring_2023.head(5)

,city_district,address,species_1,species_2,year
0,1,5121 Kenesaw St,A.Hornbeam,Wil Oak,2023
1,1,4908 Hollywood Rd,Willow Oaks,Willow Oaks,2023
2,1,4822 Erie Street,S. Magnolia,NaN,2023
3,1,5017 Laguna Road,S. Magnolia,NaN,2023
4,1,9118 Autoville Dr.,Am. Witch Hazel,NaN,2023


### Fall 2023

In [972]:
fall_2023.head(2)

,casey_tree_planting_on_private_property_fall_2023,unnamed:_1,unnamed:_2,unnamed:_3,unnamed:_4,unnamed:_5,unnamed:_6,unnamed:_7,unnamed:_8,unnamed:_9,unnamed:_10,unnamed:_11,unnamed:_12
0,Application Date; Planting on Residential Prop...,Area,District #,First and Last Name:,Email Address:,Phone Number,Address:,Preferred Contact,Consultation date,Casey Trees date planted,City inspection date,Species planted,Hold Harmless Form completed
1,2022-06-25 00:00:00,CP Estates,3,Andrew Stephens,andrewstephens11@gmail.com,202-550-5392,7512 Wellesley Dr,email,2023-06-08 00:00:00,2024-10-30 00:00:00,2023-11-17 00:00:00,American Linden,X


In [973]:
fall_2023 = format_table(fall_2023, 2023)
fall_2023.head(5)

,city_district,address,species_1,year
1,3,7512 Wellesley Dr,American Linden,2023
2,4,3711 Marlbrough Way,Red Maple,2023
3,2,4803 Ruatan St,Tulip Poplar,2023
4,1,4806 Lackawanna St,Southern Magnolia,2023
5,1,9815 52nd Pl,Tulip Poplar,2023


### Spring 2024 - 1

In [974]:
spring_2024_1.head(2)

,application_date_planting_on_residential_property,city_district,first_and_last_name:,email_address:,phone_number,address:,consultation_date,casey_pl,city_insp,species_planted,hold_harmless_form_completed
0,2023-09-17,1,Anning Cheng,anningc@yahoo.com,757-268-8522,5109 Iroquois St,2023-12-08,2024-04-05,2024-04-23,Red Maple,X
1,2023-10-08,1,Kim Tolbert,ktolbertmd@gmail.com,301-613-3496,4908 Hollywood Rd,2023-12-08,2024-04-05,2024-04-23,"Beech, Red Maple",X


In [975]:
spring_2024_1 = format_table(spring_2024_1, 2024)
spring_2024_1.head(5)

,city_district,address,city_insp,species_1,species_2,year
0,1,5109 Iroquois St,2024-04-23,Red Maple,NaN,2024
1,1,4908 Hollywood Rd,2024-04-23,Beech,Red Maple,2024
2,1,4810 Lackawanna St,2024-04-23,Dogwood,NaN,2024
3,1,5106 Lackawanna St,2024-04-24,Southern Magnolia,Swamp White Oak,2024
4,1,9735 53rd Ave,2024-04-19,Red Maple,Dogwood,2024


### Spring 2024 - 2

In [976]:
spring_2024_2.head(2)

,casey_tree_planting_on_private_property_spring_2024_may,unnamed:_1,unnamed:_2,unnamed:_3,unnamed:_4,unnamed:_5,unnamed:_6,unnamed:_7,unnamed:_8,unnamed:_9,unnamed:_10
0,Application Date; Planting on Residential Prop...,City District,First and Last Name:,Email Address:,Phone Number,Address:,Consultation date,Casey Trees date planted,City inspection date,Species planted,Hold Harmless Form completed
1,2024-02-28 00:00:00,1,Jeff Miller,jeffmillerworld@gmail.com,443-764-6083,9609 50th Place,2024-02-28 00:00:00,2024-04-10 00:00:00,2024-04-23 00:00:00,"Beech, Swamp White Oak",X


In [977]:
spring_2024_2 = format_table(spring_2024_2, 2024)
spring_2024_2.head(5)

,city_district,address,species_1,species_2,year
1,1,9609 50th Place,Beech,Swamp White Oak,2024
2,1,9540 Rhode Island Ave,Persimmon,Persimmon,2024
3,1,5121 Kennebunk Terr,Freeman Maple,NaN,2024
4,2,4811 Ruatan St,Swamp White Oak,Freemen Maple,2024
5,3,7510 Wellesley Dr,Redbud,Persimmon,2024


### Fall 2024

In [978]:
fall_2024.head(2)

,city_dist,first_and_last_name:,email_address:,phone_number:,address_of_the_tree,species_1,species_2,species_3,species_4,notes,casey_trees_date_planted,city_inspetion_date
0,1,William Crane,frunpro@outlook.com,2022465897,8611 Rhode Island Ave,American holly,Redbud,Swamp white oak,NaN,NaN,2024-11-19 00:00:00,2024-11-19 00:00:00
1,1,James Milmoe,milmoej2007@hotmail.com,4434667799,8709 48th Ave,American holly,Nuttall oak,NaN,NaN,NaN,2024-11-19 00:00:00,2024-11-19 00:00:00


In [979]:
fall_2024 = format_table(fall_2024, 2024)
fall_2024.head(5)

,city_district,address,species_1,species_2,species_3,species_4,notes,year
0,1,8611 Rhode Island Ave,American holly,Redbud,Swamp white oak,NaN,NaN,2024
1,1,8709 48th Ave,American holly,Nuttall oak,NaN,NaN,NaN,2024
2,1,9629 52nd Ave,Redbud,NaN,NaN,NaN,NaN,2024
3,1,6917 Carleton Ter,Apple serviceberry,Freeman maple,Freeman maple,Redbud,NaN,2024
4,1,9621 Narragansett Pkwy,Persimmon,Persimmon,NaN,NaN,NaN,2024


### Spring 2025

In [980]:
spring_2025.head(2)

,city_dist,first_and_last_name:,email_address:,phone_number:,address_of_the_tree,species_1,species_2,species_3,species_4,notes,casey_trees_date_planted,city_inspetion_date
0,1,Silvia Alvarado,marylicastillo@gmail.com,202-460-5615,9406 52nd Ave,Arborvitae,Arborvitae,NaN,NaN,NaN,NaN,NaN
1,1,Brian Mayers,soccerplaya37@gmail.com,443-534-4109,5202 Lackawanna St,Southern Magnolia,NaN,NaN,NaN,NaN,NaN,NaN


In [981]:
spring_2025 = format_table(spring_2025, 2024)
spring_2025.head(5)

,city_district,address,species_1,species_2,species_3,species_4,notes,year
0,1,9406 52nd Ave,Arborvitae,Arborvitae,NaN,NaN,NaN,2024
1,1,5202 Lackawanna St,Southern Magnolia,NaN,NaN,NaN,NaN,2024
2,1,9037 49th Pl,NS Holly,Redbud,NaN,NaN,NaN,2024
3,1,4712 Nantucket Rd,Am Elm,NaN,NaN,NaN,NaN,2024
4,1,5022 Geronimo St,Redbud,River Birch,NaN,NaN,NaN,2024


## Consolidate data

In [983]:
consolidated_data = pd.concat([spring_2023,fall_2023,spring_2024_1,spring_2024_2,fall_2024,spring_2025])
consolidated_data = consolidated_data[['city_district','address','species_1','species_2','species_3','species_4', 'year', 'notes']]
consolidated_data

,city_district,address,species_1,species_2,species_3,species_4,year,notes
0,1,5121 Kenesaw St,A.Hornbeam,Wil Oak,NaN,NaN,2023,NaN
1,1,4908 Hollywood Rd,Willow Oaks,Willow Oaks,NaN,NaN,2023,NaN
2,1,4822 Erie Street,S. Magnolia,NaN,NaN,NaN,2023,NaN
3,1,5017 Laguna Road,S. Magnolia,NaN,NaN,NaN,2023,NaN
4,1,9118 Autoville Dr.,Am. Witch Hazel,NaN,NaN,NaN,2023,NaN
...,...,...,...,...,...,...,...,...
28,3,5805 Chestnut Hill Rd,Loblolly Pine,Tulip Poplar,NaN,NaN,2024,NaN
29,3,7516 Wellesley Dr,NS Holly,Arborvitae,NaN,NaN,2024,NaN
30,4,8705 38th Ave,Swamp White Oak,NaN,NaN,NaN,2024,NaN
31,4,3535 Duke St,Redbud,NaN,NaN,NaN,2024,NaN


In [984]:
consolidated_data.to_csv('consolidated_data.csv')